# 1 i) Initial Setup of Classes

In [1]:
from collections import deque

class Analyst:
    def __init__(self, name, rating, workload=0, max_workload=40, available=True):
        self.name = name
        self.rating = rating  #scored 1-5
        self.workload = workload #hours assigned already default 0 
        self.max_workload = max_workload #maximum hours they can work default 40
        self.available = available # True if available for new tasks
        self.tasks = [] # List of tasks assigned to the analyst

class Task:
    def __init__(self, name, importance, effort):
        self.name = name
        self.importance = importance  # scored 1-5
        self.effort = effort # hours required to complete the task
        self.assigned_to = None

## 1 ii) dummy data

In [2]:
analysts = [
    Analyst("Alice", 5, 35, 40),
    Analyst("Bob", 4, 30, 40),
    Analyst("Charlie", 3, 20, 40),
    Analyst("Dana", 2, 10, 40),
    Analyst("Eli", 1, 5, 40),
    Analyst("Josh", 5, 0, 40)
]

tasks = [
    Task("Task1", 2, 10),
    Task("Task2", 5, 15),
    Task("Task3", 3, 20),
    Task("Task4", 4, 25),
    Task("Task5", 1, 30),
]

## 2 i) starter function

In [3]:
def assign_tasks(analysts, tasks):
    task_queue = deque(sorted(tasks, key=lambda t: (-t.importance, t.effort)))
    
    while task_queue:
        task = task_queue.popleft()
        best_analyst = None
        
        for analyst in sorted(analysts, key=lambda a: (-a.rating, a.workload)):
            if analyst.available and (analyst.workload + task.effort <= analyst.max_workload):
                best_analyst = analyst
                break
        
        if best_analyst:
            best_analyst.tasks.append(task)
            best_analyst.workload += task.effort
            best_analyst.available = best_analyst.workload < best_analyst.max_workload
            task.assigned_to = best_analyst.name
            print(f"Assigned {task.name} to {best_analyst.name}")
        else:
            print(f"No available analyst for {task.name}")
            
assign_tasks(analysts, tasks)

Assigned Task2 to Josh
Assigned Task4 to Josh
Assigned Task3 to Charlie
Assigned Task1 to Bob
Assigned Task5 to Dana


## 2 ii) reassign tasks in event of illness

In [4]:
analysts = [
    Analyst("Alice", 5, 35, 40),
    Analyst("Bob", 4, 30, 40),
    Analyst("Charlie", 3, 20, 40),
    Analyst("Dana", 2, 10, 40),
    Analyst("Eli", 1, 5, 40),
    Analyst("Josh", 5, 0, 40)
]

tasks = [
    Task("Task1", 2, 10),
    Task("Task2", 5, 15),
    Task("Task3", 3, 20),
    Task("Task4", 4, 25),
    Task("Task5", 1, 30),
]

analysts.pop(2) #Charlie is ill
assign_tasks(analysts,tasks)

Assigned Task2 to Josh
Assigned Task4 to Josh
Assigned Task3 to Dana
Assigned Task1 to Bob
Assigned Task5 to Eli


## 3 i) New rule: Analysts may not attempt tasks with higher importance level than their analyst level

#### Analysts are now dicts not lists to allow task reassignment in 3 ii)

In [5]:
def assign_tasks_max_level(analysts_dict:dict, tasks:list):
    analysts = list(analysts_dict.values())
    task_queue = deque(sorted(tasks, key=lambda t: (-t.importance, t.effort)))
    unnasigned_tasks=[]
    
    while task_queue:
        task = task_queue.popleft()
        best_analyst = None
        
        for analyst in sorted(analysts, key=lambda a: (-a.rating, a.workload)):
            if analyst.available and (analyst.workload + task.effort <= analyst.max_workload):
                if analyst.rating >= task.importance:
                    best_analyst = analyst
                else:
                    print(f"No available analyst level {task.importance} or above for {task.name}")
                    unnasigned_tasks.append(task)
                break
        #condition to find max hours left for analyst
        hours_remaining = deque(sorted(analysts, key=lambda a: (- a.max_workload + a.workload)))
        
        if best_analyst:
            best_analyst.tasks.append(task)
            best_analyst.workload += task.effort
            best_analyst.available = best_analyst.workload < best_analyst.max_workload
            task.assigned_to = best_analyst.name
            print(f"Assigned {task.name} to {best_analyst.name}")
        elif task.effort > (hours_remaining[0].max_workload-hours_remaining[0].workload):
            print(f"""No available analyst for {task.name}, The task requires {task.effort} hours, however the employee with 
                  the most available hours is {hours_remaining[0].name} who has {hours_remaining[0].max_workload-hours_remaining[0].workload} hours available, 
                  consider a temporary contractor""")
            unnasigned_tasks.append(task)
    if len(unnasigned_tasks) > 0:
        print(f"WARNING: Tasks unable to be assigned are {[task.name for task in unnasigned_tasks]}")

## 3 ii) task realocattion - if analyst unavailable, keeps all analysts with existing tasks where possible

In [6]:
def reassign_tasks_unavailability(analysts_dict:dict ,unavailable_analyst:str):
    analysts_dict[unavailable_analyst].available = False
    tasks_for_reassignment=analysts_dict[unavailable_analyst].tasks
    print('tasks for reassignment:',[task.name for task in tasks_for_reassignment])
    analysts = list(analysts_dict.values())
    task_queue = deque(sorted(tasks_for_reassignment, key=lambda t: (-t.importance, t.effort)))
    unnasigned_tasks=[]
    
    while task_queue:
        task = task_queue.popleft()
        best_analyst = None
        
        for analyst in sorted(analysts, key=lambda a: (-a.rating, a.workload)):
            if analyst.available and (analyst.workload + task.effort <= analyst.max_workload):
                if analyst.rating >= task.importance:
                    best_analyst = analyst
                else:
                    print(f"No available analyst level {task.importance} or above for {task.name}")
                    unnasigned_tasks.append(task)
                break
        #condition to find max hours left for analyst
        hours_remaining = deque(sorted(analysts, key=lambda a: (- a.max_workload + a.workload)))
        
        if best_analyst:
            best_analyst.tasks.append(task)
            best_analyst.workload += task.effort
            best_analyst.available = best_analyst.workload < best_analyst.max_workload
            task.assigned_to = best_analyst.name
            print(f"Assigned {task.name} to {best_analyst.name}")
        elif task.effort > (hours_remaining[0].max_workload-hours_remaining[0].workload):
            print(f"""No available analyst for {task.name}, The task requires {task.effort} hours, however the employee with 
                  the most available hours is {hours_remaining[0].name} who has {hours_remaining[0].max_workload-hours_remaining[0].workload} hours available, 
                  consider a temporary contractor""")
            unnasigned_tasks.append(task)
    if len(unnasigned_tasks) > 0:
        print(f"WARNING: Tasks unable to be assigned are {[task.name for task in unnasigned_tasks]}")
        
analysts_dict = {
    "Alice": Analyst("Alice", 5, 0, 40),
    "Bob": Analyst("Bob", 4, 0, 40),
    "Charlie": Analyst("Charlie", 3, 0, 40),
    "Dana": Analyst("Dana", 2, 0, 40),
    "Eli": Analyst("Eli", 1, 0, 40),
    "Josh":Analyst("Josh", 5, 0, 40)
}
assign_tasks_max_level(analysts_dict,tasks)
reassign_tasks_unavailability(analysts_dict,"Josh")

Assigned Task2 to Alice
Assigned Task4 to Josh
Assigned Task3 to Alice
Assigned Task1 to Josh
Assigned Task5 to Bob
tasks for reassignment: ['Task4', 'Task1']
No available analyst level 4 or above for Task4
Assigned Task1 to Bob


### To do list:

##### Recursive feature which only reassignes tasks when neccecary
##### Optimise for minimal time wastage/reassignment

## 4 i) create single layer recursive function 

In [7]:
def reassign_tasks_reprioritise(analysts_dict:dict ,analyst:str,task:object):
    tasks_for_reassignment=analysts_dict[analyst].tasks
    analysts_dict[analyst].workload=0
    analysts_dict[analyst].tasks = [] # List of tasks assigned to the analyst
    tasks_for_reassignment.append(task)
    print('tasks for reassignment:',[task.name for task in tasks_for_reassignment])
    analysts = list(analysts_dict.values())
    task_queue = deque(sorted(tasks_for_reassignment, key=lambda t: (-t.importance, t.effort)))
    unnasigned_tasks=[]
    
    while task_queue:
        task = task_queue.popleft()
        best_analyst = None
        
        for analyst in sorted(analysts, key=lambda a: (-a.rating, a.workload)):
            if analyst.available and (analyst.workload + task.effort <= analyst.max_workload):
                if analyst.rating >= task.importance:
                    best_analyst = analyst
                else:
                    print(f"No available analyst level {task.importance} or above for {task.name}")
                    unnasigned_tasks.append(task)
                break
        #condition to find max hours left for analyst
        hours_remaining = deque(sorted(analysts, key=lambda a: (- a.max_workload + a.workload)))
        
        if best_analyst:
            best_analyst.tasks.append(task)
            best_analyst.workload += task.effort
            best_analyst.available = best_analyst.workload < best_analyst.max_workload
            task.assigned_to = best_analyst.name
            print(f"Assigned {task.name} to {best_analyst.name}")
        elif task.effort > (hours_remaining[0].max_workload-hours_remaining[0].workload):
            print(f"""No available analyst for {task.name}, The task requires {task.effort} hours, however the employee with 
                  the most available hours is {hours_remaining[0].name} who has {hours_remaining[0].max_workload-hours_remaining[0].workload} hours available, 
                  consider a temporary contractor""")
            unnasigned_tasks.append(task)
    if len(unnasigned_tasks) > 0:
        print(f"WARNING: Tasks unable to be assigned are {[task.name for task in unnasigned_tasks]}..Attempting new loop of recursive reassignment")
        sorted_reassignment = deque(sorted(unnasigned_tasks, key=lambda t: (-t.importance)))
        potential_analysts=[]
        for analyst in analysts:
            if analyst.available and (analyst.rating >= sorted_reassignment[0].importance):
                potential_analysts.append(analyst)
        print('Potential analysts:',[analyst.name for analyst in potential_analysts])
        for analyst in potential_analysts:            
            for task in analyst.tasks:             
                print('reached') 
                if task.importance < sorted_reassignment[0].importance:
                    print('Attempting recursive reassignment, currently reasigning tasks for', analyst.name)
                    reassign_tasks_reprioritise(analysts_dict,analyst.name,sorted_reassignment[0])
                    break
        
        
    

In [8]:
def reassign_tasks_unavailability(analysts_dict:dict ,unavailable_analyst:str):
    print('Unavailable analyst:', analysts_dict[unavailable_analyst].name)
    analysts_dict[unavailable_analyst].available = False
    tasks_for_reassignment=analysts_dict[unavailable_analyst].tasks
    print('Tasks for reassignment:',[task.name for task in tasks_for_reassignment])
    analysts = list(analysts_dict.values())
    task_queue = deque(sorted(tasks_for_reassignment, key=lambda t: (-t.importance, t.effort)))
    unnasigned_tasks=[]
    
    while task_queue:
        task = task_queue.popleft()
        best_analyst = None
        
        for analyst in sorted(analysts, key=lambda a: (-a.rating, a.workload)):
            if analyst.available and (analyst.workload + task.effort <= analyst.max_workload):
                if analyst.rating >= task.importance:
                    best_analyst = analyst
                else:
                    print(f"No available analyst level {task.importance} or above for {task.name}")
                    unnasigned_tasks.append(task)
                break
        #condition to find max hours left for analyst
        hours_remaining = deque(sorted(analysts, key=lambda a: (- a.max_workload + a.workload)))
        
        if best_analyst:
            best_analyst.tasks.append(task)
            best_analyst.workload += task.effort
            best_analyst.available = best_analyst.workload < best_analyst.max_workload
            task.assigned_to = best_analyst.name
            print(f"Assigned {task.name} to {best_analyst.name}")
        elif task.effort > (hours_remaining[0].max_workload-hours_remaining[0].workload):
            print(f"""No available analyst for {task.name}, The task requires {task.effort} hours, however the employee with 
                  the most available hours is {hours_remaining[0].name} who has {hours_remaining[0].max_workload-hours_remaining[0].workload} hours available, 
                  consider a temporary contractor""")
            unnasigned_tasks.append(task)
    if len(unnasigned_tasks) > 0:
        print(f"WARNING: Tasks unable to be assigned are: {[task.name for task in unnasigned_tasks]}, attempting recursive reassignment..")
        sorted_reassignment = deque(sorted(unnasigned_tasks, key=lambda t: (-t.importance)))
        potential_analysts=[]
        for analyst in analysts:
            if analyst.available and (analyst.rating >= sorted_reassignment[0].importance):
                potential_analysts.append(analyst)
        print('Potential analysts:',[analyst.name for analyst in potential_analysts])
        for analyst in potential_analysts:            
            for task in analyst.tasks:             
                if task.importance < sorted_reassignment[0].importance:
                    print('Attempting recursive reassignment, currently reasigning tasks for', analyst.name)
                    reassign_tasks_reprioritise(analysts_dict,analyst.name,sorted_reassignment[0])
                    break
                
        
        
        
analysts_dict = {
    "Alice": Analyst("Alice", 5, 0, 40), #temporary fix to make alice available
    "Bob": Analyst("Bob", 4, 0, 40),
    "Charlie": Analyst("Charlie", 3, 0, 40),
    "Dana": Analyst("Dana", 2, 0, 40),
    "Eli": Analyst("Eli", 1, 0, 40),
    "Freddie":Analyst("Freddie", 5, 0, 40),
    "Graham":Analyst("Graham",5,0,20) #semi-retired works part time
}

tasks = [
    Task("Task1", 2, 10),
    Task("Task2", 5, 15),
    Task("Task3", 3, 20),
    Task("Task4", 4, 25),
    Task("Task5", 1, 30),
    Task("Task6", 5, 20)
]
assign_tasks_max_level(analysts_dict,tasks)
reassign_tasks_unavailability(analysts_dict,"Freddie")

Assigned Task2 to Alice
Assigned Task6 to Freddie
Assigned Task4 to Alice
Assigned Task3 to Graham
Assigned Task1 to Freddie
Assigned Task5 to Bob
Unavailable analyst: Freddie
Tasks for reassignment: ['Task6', 'Task1']
No available analyst level 5 or above for Task6
Assigned Task1 to Bob
Potential analysts: []


## No available analysts due to way class is set up so modifying class - nearly ready to switch to .py

#### All my functions are fairly similar could likely merge into a class...

In [9]:
class Analyst:
    def __init__(self, name, rating, workload=0, max_workload=40, available=True):
        self.name = name
        self.rating = rating  #scored 1-5
        self.workload = workload #hours assigned already default 0 
        self.max_workload = max_workload #maximum hours they can work default 40
        self.available = available # Available to take on extra work
        self.absent = False #able to complete any work
        self.tasks = [] # List of tasks assigned to the analyst

class Task:
    def __init__(self, name, importance, effort):
        self.name = name
        self.importance = importance  # scored 1-5
        self.effort = effort # hours required to complete the task
        self.assigned_to = None

In [10]:
def reassign_tasks_reprioritise(analysts_dict:dict ,analyst:str,task:object):
    tasks_for_reassignment=analysts_dict[analyst].tasks
    analysts_dict[analyst].workload=0
    analysts_dict[analyst].tasks = [] # List of tasks assigned to the analyst
    tasks_for_reassignment.append(task)
    print('tasks for reassignment:',[task.name for task in tasks_for_reassignment])
    analysts = list(analysts_dict.values())
    task_queue = deque(sorted(tasks_for_reassignment, key=lambda t: (-t.importance, t.effort)))
    unnasigned_tasks=[]
    
    while task_queue:
        task = task_queue.popleft()
        best_analyst = None
        
        for analyst in sorted(analysts, key=lambda a: (-a.rating, a.workload)):
            if not analyst.absent and (analyst.workload + task.effort <= analyst.max_workload):
                if analyst.rating >= task.importance:
                    best_analyst = analyst
                else:
                    print(f"No available analyst level {task.importance} or above for {task.name}")
                    unnasigned_tasks.append(task)
                break
        #condition to find max hours left for analyst
        hours_remaining = deque(sorted(analysts, key=lambda a: (- a.max_workload + a.workload)))
        
        if best_analyst:
            best_analyst.tasks.append(task)
            best_analyst.workload += task.effort
            best_analyst.available = best_analyst.workload < best_analyst.max_workload
            task.assigned_to = best_analyst.name
            print(f"Assigned {task.name} to {best_analyst.name}")
        elif task.effort > (hours_remaining[0].max_workload-hours_remaining[0].workload):
            print(f"""No available analyst for {task.name}, The task requires {task.effort} hours, however the employee with 
                  the most available hours is {hours_remaining[0].name} who has {hours_remaining[0].max_workload-hours_remaining[0].workload} hours available, 
                  consider a temporary contractor""")
            unnasigned_tasks.append(task)
    if len(unnasigned_tasks) > 0:
        print(f"WARNING: Tasks unable to be assigned are {[task.name for task in unnasigned_tasks]}..Attempting new loop of recursive reassignment")
        sorted_reassignment = deque(sorted(unnasigned_tasks, key=lambda t: (-t.importance)))
        potential_analysts=[]
        for analyst in analysts:
            if analyst.available and (analyst.rating >= sorted_reassignment[0].importance):
                potential_analysts.append(analyst)
        print('Potential analysts:',[analyst.name for analyst in potential_analysts])
        for analyst in potential_analysts:            
            for task in analyst.tasks:              
                if task.importance < sorted_reassignment[0].importance:
                    print('Attempting recursive reassignment, currently reasigning tasks for', analyst.name)
                    reassign_tasks_reprioritise(analysts_dict,analyst.name,sorted_reassignment[0])
                    break
        
        
    

In [11]:
def reassign_tasks_unavailability(analysts_dict:dict ,unavailable_analyst:str):
    print('Unavailable analyst:', analysts_dict[unavailable_analyst].name)
    analysts_dict[unavailable_analyst].available = False
    analysts_dict[unavailable_analyst].absent = True
    tasks_for_reassignment=analysts_dict[unavailable_analyst].tasks
    print('Tasks for reassignment:',[task.name for task in tasks_for_reassignment])
    analysts = list(analysts_dict.values())
    task_queue = deque(sorted(tasks_for_reassignment, key=lambda t: (-t.importance, t.effort)))
    unnasigned_tasks=[]
    
    while task_queue:
        task = task_queue.popleft()
        best_analyst = None
        
        for analyst in sorted(analysts, key=lambda a: (-a.rating, a.workload)):
            if analyst.available and (analyst.workload + task.effort <= analyst.max_workload):
                if analyst.rating >= task.importance:
                    best_analyst = analyst
                else:
                    print(f"No available analyst level {task.importance} or above for {task.name}")
                    unnasigned_tasks.append(task)
                break
        #condition to find max hours left for analyst
        hours_remaining = deque(sorted(analysts, key=lambda a: (- a.max_workload + a.workload)))
        
        if best_analyst:
            best_analyst.tasks.append(task)
            best_analyst.workload += task.effort
            best_analyst.available = best_analyst.workload < best_analyst.max_workload
            task.assigned_to = best_analyst.name
            print(f"Assigned {task.name} to {best_analyst.name}")
        elif task.effort > (hours_remaining[0].max_workload-hours_remaining[0].workload):
            print(f"""No available analyst for {task.name}, The task requires {task.effort} hours, however the employee with 
                  the most available hours is {hours_remaining[0].name} who has {hours_remaining[0].max_workload-hours_remaining[0].workload} hours available, 
                  consider a temporary contractor""")
            unnasigned_tasks.append(task)
    if len(unnasigned_tasks) > 0:
        print(f"WARNING: Tasks unable to be assigned are: {[task.name for task in unnasigned_tasks]}, attempting recursive reassignment..")
        sorted_reassignment = deque(sorted(unnasigned_tasks, key=lambda t: (-t.importance)))
        potential_analysts=[]
        for analyst in analysts:
            if not analyst.absent and (analyst.rating >= sorted_reassignment[0].importance):
                potential_analysts.append(analyst)
        print('Potential analysts:',[analyst.name for analyst in potential_analysts])
        for analyst in potential_analysts:            
            for task in analyst.tasks:             
                if task.importance < sorted_reassignment[0].importance:
                    print('Attempting recursive reassignment, currently reasigning tasks for', analyst.name)
                    reassign_tasks_reprioritise(analysts_dict,analyst.name,sorted_reassignment[0])
                    break
                
        
        
        
analysts_dict = {
    "Alice": Analyst("Alice", 5, 0, 40), 
    "Bob": Analyst("Bob", 4, 0, 40),
    "Charlie": Analyst("Charlie", 3, 0, 40),
    "Dana": Analyst("Dana", 2, 0, 40),
    "Eli": Analyst("Eli", 1, 0, 40),
    "Freddie":Analyst("Freddie", 5, 0, 40),
    "Graham":Analyst("Graham",5,0,20) #semi-retired works part time
}

tasks = [
    Task("Task1", 2, 10),
    Task("Task2", 5, 15),
    Task("Task3", 3, 20),
    Task("Task4", 4, 25),
    Task("Task5", 1, 30),
    Task("Task6", 5, 20)
]
assign_tasks_max_level(analysts_dict,tasks)
reassign_tasks_unavailability(analysts_dict,"Freddie")

Assigned Task2 to Alice
Assigned Task6 to Freddie
Assigned Task4 to Alice
Assigned Task3 to Graham
Assigned Task1 to Freddie
Assigned Task5 to Bob
Unavailable analyst: Freddie
Tasks for reassignment: ['Task6', 'Task1']
No available analyst level 5 or above for Task6
Assigned Task1 to Bob
Potential analysts: ['Alice', 'Graham']
Attempting recursive reassignment, currently reasigning tasks for Alice
tasks for reassignment: ['Task2', 'Task4', 'Task6']
Assigned Task2 to Alice
Assigned Task6 to Alice
No available analyst level 4 or above for Task4
Potential analysts: ['Alice']
Attempting recursive reassignment, currently reasigning tasks for Graham
tasks for reassignment: ['Task3', 'Task6']
Assigned Task6 to Graham
Assigned Task3 to Charlie


## Working solution for this particular problem :)

#### Still to do...

#### Anti infinate-recursion mechanisms - maybe some way of only calling each person once in recursive function - if so probably need better sorting mechanism for order recursion called

#### Convert functions to class

#### Potential analysts: 'Alice' Attempting recursive reassignment, currently reasigning tasks for Graham - understand why this happens

#### Some nice final output something like:

Assigned Task2 to Alice

Assigned Task6 to Freddie

Assigned Task4 to Alice

Assigned Task3 to Graham

Assigned Task1 to Freddie

Assigned Task5 to Bob

#### Explore extra things like adding in an additional department with similar training who have already got some hours planned in their own department but if capacity for 10 more hours draft them in to new department.

#### Explore optimising for minimum time wasteage



## Below is an example showing where improvement is needed

#### Alice unable to take on task 6 and becomes stuck in loop - this could be prime case to call on neighbouring department as too busy to do themselves

In [14]:
analysts_dict = {
    "Alice": Analyst("Alice", 5, 0, 40),
    "Bob": Analyst("Bob", 4, 0, 35),
    "Charlie": Analyst("Charlie", 3, 0, 40),
    "Dana": Analyst("Dana", 2, 0, 30),
    "Eli": Analyst("Eli", 1, 0, 40),
    "Freddie": Analyst("Freddie", 5, 0, 40),
    "Graham": Analyst("Graham", 5, 0, 20),
    "Hannah": Analyst("Hannah", 4, 0, 40),
    "Ivan": Analyst("Ivan", 2, 0, 25)
}

tasks = [
    # Level 1 tasks (small, numerous)
    Task("Task1", 1, 5),
    Task("Task2", 1, 5),
    Task("Task3", 1, 5),
    Task("Task4", 1, 10),
    Task("Task5", 1, 5),
    Task("Task6", 1, 10),
    Task("Task7", 1, 5),
    Task("Task8", 1, 5),

    # Level 2 tasks (still small)
    Task("Task9", 2, 10),
    Task("Task10", 2, 10),
    Task("Task11", 2, 10),
    Task("Task12", 2, 15),
    Task("Task13", 2, 10),

    # Level 3–5 tasks (fewer, more demanding)
    Task("Task14", 3, 15),
    Task("Task15", 3, 20),
    Task("Task16", 4, 20),
    Task("Task17", 4, 25),
    Task("Task18", 5, 20),
    Task("Task20", 5, 30)
]


assign_tasks_max_level(analysts_dict,tasks)
reassign_tasks_unavailability(analysts_dict,"Freddie")

Assigned Task18 to Alice
Assigned Task20 to Freddie
Assigned Task16 to Graham
Assigned Task17 to Bob
Assigned Task14 to Alice
Assigned Task15 to Hannah
Assigned Task9 to Freddie
Assigned Task10 to Hannah
Assigned Task11 to Bob
Assigned Task13 to Hannah
Assigned Task12 to Charlie
Assigned Task1 to Alice
Assigned Task2 to Charlie
Assigned Task3 to Charlie
Assigned Task5 to Charlie
Assigned Task7 to Charlie
Assigned Task8 to Charlie
Assigned Task4 to Dana
Assigned Task6 to Ivan
Unavailable analyst: Freddie
Tasks for reassignment: ['Task20', 'Task9']
No available analyst level 5 or above for Task20
Assigned Task9 to Dana
Potential analysts: ['Alice', 'Graham']
Attempting recursive reassignment, currently reasigning tasks for Alice
tasks for reassignment: ['Task18', 'Task14', 'Task1', 'Task20']
Assigned Task18 to Alice
No available analyst level 5 or above for Task20
Assigned Task14 to Alice
Assigned Task1 to Alice
Potential analysts: []
Attempting recursive reassignment, currently reasigni